# Scrape Jobindex

## Import packages

In [2]:
# import packages
import requests, json
import pandas as pd, numpy as np
import time, os

## Get web data

Procedure
collect the first page
save the totalresultcount for later usage


In [170]:
# the basic url
# https://www.jobindex.dk/jobsoegning.json?q=&jobage=archive&mindate=20180524&maxdate=20180824&page=2
url = 'https://www.jobindex.dk/jobsoegning.json'


# the request session
session = requests.session()
session.headers['email'] = 'pot@potweb.dk' 
session.headers['name'] = 'Peter Ottosen'
#session.headers['User-Agent'] = '' # sometimes you need to pose as another agent...
session.headers

# container for jobpostings
jobpostings = []
done = []

global last_t
last_t = time.time()

# function to create the next link
def create_link(offset_) :
    mindate = 20080101
    maxdate = 20091231
    addit = '?q=&jobage=archive&mindate=' + str(mindate) + '&maxdate=' + str(maxdate) + '&page=' + str(offset_)
    return url + addit

def ratelimit():
    time.sleep(2) # sleep one second.

# Reliable requests
def get(url,iterations=10,check_function=lambda x: x.ok):
    """This module ensures that your script does not crash from connection errors.
        that you limit the rate of your calls
        that you have some reliability check
        iterations : Define number of iterations before giving up. 
        exceptions: Define which exceptions you accept, default is all. 
    """
    for iteration in range(iterations):
        try:
            # add ratelimit function call here
            ratelimit() # !!
            response = session.get(url)
            if response.ok:
                return response # if succesful it will end the iterations here
        except exceptions as e: #  find exceptions in the request library requests.exceptions
            print(e) # print or log the exception message.
    return None # your code will purposely crash if you don't create a check function later.

def log_function(url,response,error_check=lambda x: x.ok,separator=',') :
    global last_t
    logfilepath = r"C:\Notebooks\group19\Eksamensprojekt/log_jobnet.csv"
    if os.path.isfile(logfilepath) :
        f_log = open(logfilepath,'a')
    else:
        f_log = open(logfilepath,'w') # define logfile, remember not to overwrite it.
        # write columns to be used, basic ones are, servertime, deltaT since last call, url, success of request, 
        header = ['serverTime','deltaT','url','success','length','path']
        f_log.write(','.join(header)+'\n')
    #### Update timing info ####
    t = time.time()
    delta_t = t-last_t # calculate time since last call
    last_t = t# update last call time
    #### meta data ### 
    success = error_check(response)
    if success: # if call is successfull we add it to the done container
        done.append(url)
    if response.ok:
        length = len(response.text)
    else:
        length = 0
    row = [t,delta_t,url,success,length,path]
    f_log.write(separator.join(map(str,row))+'\n')
    f_log.close()


# just to get the number of results
offset = 1
count = 0
r = get(create_link(offset))
d=json.loads(r.text)

#ResultCount=d['TotalResultCount']
#jp_keys = d['JobPositionPostings'][0].keys()

for pages in range(2, 3) : # pagineringen
    offset = pages
    link = create_link(offset)
    if link in done: #check if you have already downloaded the link
        continue
    count +=1
    r = get(link)
    d=json.loads(r.text)
#    jobpostings.append(d['JobPositionPostings'])

    #define path 
    base_path = r"C:\Notebooks\group19\Eksamensprojekt/jobindex.txt"
    path = base_path      # use the link count as a filename
    if r.ok:
        if os.path.isfile(path) :
            f = open(path,'a')
        else:
            f = open(path,'w') # define logfile, remember not to overwrite it.
            
        html = r.text
        try :
            f.write(html)
        except UnicodeEncodeError :
            print("Der opstod en UnicodeEncodeError ved skrivning af side", str(pages))
        else :
            pass
        f.write('\n')
        f.close()
    # run your log function
    log_function(link, r)



In [178]:
from bs4 import BeautifulSoup
import re

#print(d.keys())

rl = d['result_list_box_html']

lineshift = re.compile('\n')
#rl2 = lineshift.sub('<br>', rl)
#print(rl2)
soup = BeautifulSoup(rl,'lxml')

tabel = soup.find('div', attrs={'class':'results component--default'})

# Jobindex' importerede annoncer
andre = re.compile('r[\d]+')
tabel1 = tabel.findAll('div', attrs={'data-beacon-tid':andre})
#print(tabel1)

# Jobindex' egne annoncer
egne = re.compile('h[\d]+')
tabel2 = tabel.findAll('div', attrs={'data-beacon-tid':egne})
#print(tabel2)


beskre = []

# tag ul skal med 
for l in tabel2 :
    g = l.findAll('p')
    for m in g :
        beskre.append(m.text)
print(beskre)


#p=re.compile('"')
#for t_ in tabel2 :
#    beskr.append(lineshift.sub("", t_.text))
#
#beskre2 = []
#for t in beskr :
#    beskr2.append(t.split(sep='    ')[1])
#for t in beskr2 :
#    print(t)

    
    
#hyper = []
#x1 = tabel.findAll('a', attrs={'href':True}) 
#for x in x1 :
#    hyper.append(x.)
#    
#print(x1)



['\nChartis, København', ' Teoretisk velfunderet og initiativrig underwriter søges af international forsikringskoncern. ', ' Du skal beskæftige dig med analyse og risikovurdering indenfor erhvervs- og produktansvarsforsikring, autoforsikring og arbejdsskadeforsikring i segmentet for mindre og mellemstore virksomheder. Herudover skal du forhandle forsikringsaftaler, lave skadeovervågning, formulere forsikringsvilkår og lave opsøgende salgsarbejde, primært via forsikringsmæglere. ', ' Vi lægger vægt på, at du har entreprenør ånd, er udadvendt, målbevidst, har mod på komplekse problemstillinger, og ikke mindst er du en teamplayer! ', '\nBilka, København', 'Som salgsassistentelev bliver du med det sammen en del af et velfungerende team. Du får kundekontakt fra første dag og du får hurtigt ansvar for din egen varegruppe. ', ' Uddannelsen tager 2 år og er en kombination af 8 ugers kursusophold samt praktik i varehuset. I løbet af de 2 år lærer du grundprincipperne inden for kundeservice, var

## Jobs hos andre

In [173]:
# udled jobs andre
jobs = []
j = tabel.findAll('strong') 
for l in j :
    jobs.append(l.text)
print(jobs)

#udled jobbeskrivelse andre
beskr = []
p=re.compile('"')
for t_ in tabel1 :
    beskr.append(lineshift.sub("", t_.text))

beskr2 = []
for t in beskr :
    beskr2.append(t.split(sep='    ')[1])
for t in beskr2 :
    print(t)


['Sales/Customer Business Development Account Manager Intern (English), Copenhagen Area', 'Sygeplejerske, medicinsk gastroenterologisk sengeafsnit D9, Medicinsk afdeling', 'Lægesekretær', 'Timelærer i billedkunst', 'Erhvervsorienteret advokat og advokatfuldmægtige', 'Social- og sundhedsassistent eller plejer til Retspsykiatrisk Afdeling']


## Oversigt over data

In [95]:
print("Pages read:", len(jobpostings), type(jobpostings))
print("Postings per page:", len(jobpostings[1]), type(jobpostings[0]))
print("Number of variables in posting:", len(jobpostings[0][0]), type(jobpostings[0][0]))
for key in jp_keys :
    print(key, ":", jobpostings[0][0][key])

Pages read: 3 <class 'list'>
Postings per page: 20 <class 'list'>
Number of variables in posting: 42 <class 'dict'>
AutomatchType : 0
Weight : 1.0
Title : køkkenmedhjælper
JobHeadline : køkkenmedhjælper
Presentation : <p>Mama Rosa s&oslash;ger lige i &oslash;jeblikket 2 engagerede dygtige k&oslash;kkenmedhj&aelig;lper/pizzabagere med godt kendskab til det internationale k&oslash;kken.</p>
<p>Vi har fokus p&aring; k
HiringOrgName : MAMA ROSA COPENHAGEN APS
WorkPlaceAddress : Østergade 57
WorkPlacePostalCode : 1100
WorkPlaceCity : København K
WorkPlaceOtherAddress : False
WorkPlaceAbroad : False
WorkPlaceNotStatic : False
UseWorkPlaceAddressForJoblog : True
PostingCreated : 2018-08-23T00:00:00
LastDateApplication : 2018-10-18T00:00:00
AssignmentStartDate : 0001-01-01T00:00:00
IsHotjob : False
IsExternal : False
Url : https://job.jobnet.dk/CV/FindWork/JobDetail/4866851
Region : Hovedstaden og Bornholm
Municipality : København
Country : Danmark
PostalCode : 1100
PostalCodeName : None
JobAn

In [7]:
len(done)

156